In [11]:
# Importo librerías necesarias
import pandas as pd
import numpy as np

In [12]:
# Cargo archivos csv
df_amazon = pd.read_csv("amazon_prime_titles.csv")
df_disney = pd.read_csv("disney_plus_titles.csv")
df_hulu = pd.read_csv("hulu_titles.csv")
df_netflix = pd.read_csv("netflix_titles.csv")

In [13]:
# Creo una lista con todos los archivos csv de ratings
archivos_ratings = ["ratings/1.csv","ratings/2.csv","ratings/3.csv","ratings/4.csv","ratings/5.csv","ratings/6.csv","ratings/7.csv","ratings/8.csv"]

# Creo una lista para almacenar todos los dataframes
dataframes = []

# Cargo todos los archivos csv en un dataframe con un bucle for
for archivo in archivos_ratings:
    df = pd.read_csv(archivo)
    dataframes.append(df)

# Concateno los dataframes en uno solo
df_score = pd.concat(dataframes)

In [14]:
# Cambio el nombre de la columna 'rating'
df_score = df_score.rename(columns = {'rating':'score'})

In [15]:
# Creo un nuevo campo 'id' para cada dataframe
df_amazon['id'] = 'a' + df_amazon['show_id'].astype(str)
df_disney['id'] = 'd' + df_disney['show_id'].astype(str)
df_hulu['id'] = 'h' + df_hulu['show_id'].astype(str)
df_netflix['id'] = 'n' + df_netflix['show_id'].astype(str)

In [16]:
# Reemplazo los valores nulos del campo rating por el string 'G'
df_amazon.rating.fillna(value='G', inplace=True)
df_disney.rating.fillna(value='G', inplace=True)
df_hulu.rating.fillna(value='G', inplace=True)
df_netflix.rating.fillna(value='G', inplace=True)

In [17]:
# Convierto las columnas "date_added" a formato de fecha
df_amazon['date_added'] = pd.to_datetime(df_amazon['date_added'])
df_disney['date_added'] = pd.to_datetime(df_disney['date_added'])
df_hulu['date_added'] = pd.to_datetime(df_hulu['date_added'])
df_netflix['date_added'] = pd.to_datetime(df_netflix['date_added'])

In [18]:
# Cambio el formato de fechas a 'AAAA-MM-DD'
df_amazon['date_added'] = df_amazon['date_added'].dt.strftime('%Y-%m-%d')
df_disney['date_added'] = df_disney['date_added'].dt.strftime('%Y-%m-%d')
df_hulu['date_added'] = df_hulu['date_added'].dt.strftime('%Y-%m-%d')
df_netflix['date_added'] = df_netflix['date_added'].dt.strftime('%Y-%m-%d')

In [19]:
# Convierte la columna de fechas a objetos de fecha y hora
df_score['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Cambia el formato de las fechas a 'YYYY/MM/DD'
df_score['timestamp'] = df_score['timestamp'].dt.strftime('%Y-%m-%d')

In [20]:
# Convierto los valores de las columnas a string
df_amazon = df_amazon.astype(str)
df_disney = df_disney.astype(str)
df_hulu = df_hulu.astype(str)
df_netflix = df_netflix.astype(str)

In [21]:
# Convierto todos los valores de texto en las columnas a minúscula
df_amazon = df_amazon.apply(lambda x: x.str.lower())
df_disney = df_disney.apply(lambda x: x.str.lower())
df_hulu = df_hulu.apply(lambda x: x.str.lower())
df_netflix = df_netflix.apply(lambda x: x.str.lower())

In [22]:
# Uno los cuatro dataframes de las plataformas en uno solo
dfs = [df_amazon, df_disney, df_hulu, df_netflix]
df_plataformas = pd.concat(dfs, ignore_index=True)

In [23]:
# Covierto el campo duration en dos campos: 'duration_int' y 'duration_type'
df_plataformas['duration_int'] = df_plataformas['duration'].str.extract('(\d+)', expand=False)
df_plataformas['duration_type'] = df_plataformas['duration'].str.extract('(\D+)', expand=False)

In [24]:
# Elimino valores nulos para poder cambiar el formato
df_plataformas['duration_int'].fillna(0, inplace=True)

# Convierto la columna 'duration_int' a formato integer
df_plataformas['duration_int'] = df_plataformas['duration_int'].astype(int)


In [25]:
# Elimino la columna 'duration'
df_plataformas.drop('duration', axis=1, inplace=True)

In [26]:
# Creo la columna 'plataform'
condicion =[
    df_plataformas['id'].str.startswith('a'),
    df_plataformas['id'].str.startswith('d'),
    df_plataformas['id'].str.startswith('h'),
    df_plataformas['id'].str.startswith('n')
]
#Valores que quiero en mi nuevo campo "plataform" si se cumplen las condiciones
valor= ["amazon", "disney_plus", "hulu", "netflix"]
df_plataformas['plataform'] = np.select(condicion, valor)

In [42]:
# Uno los dos dataframes vinculando los id
df_final = pd.merge(df_plataformas, df_score, left_on='id', right_on='movieId', how='inner')

KeyboardInterrupt: 

In [28]:
# Guardao los dataframes en un archivo CSV
df_plataformas.to_csv('plataformas.csv', index=False)
df_score.to_csv('scores.csv', index=False)